# TUTORIAL:
# Julia for technical computing

This tutorial is a collection of notebooks aimed at Matlab, Python and C++ users who are already familiar with technical computing. The tutorial is by no means a substitute for the [Julia manual](http://docs.julialang.org/en/release-0.4/manual). In fact, in many places we will simply refer to the manual. Or, for that matter, to [Github issues](https://github.com/JuliaLang/julia/issues) or the [Julia-users](https://groups.google.com/forum/#!forum/julia-users) mailing list.

With the manual at hand, in these notebooks we introduce a *Julian* way of programming. Writing efficient Julia code revolves around designing suitable **types**, enabling **type inference** and using **dynamic multiple dispatch** to our advantage. If you have never heard about these things, don't worry: neither had I! The concepts are easy, we'll see. Types can be loosely compared to classes in an object-oriented programming (OOP) language, but they are sufficiently different that they take some getting used to. The goal of these notebooks is simply to get used to them quicker! In doing so, we intend to illustrate why this combination of type inference and multiple dispatch is a great match for the challenges in technical computing.

## CHAPTER 0. Two motivating examples: why Julia?

We introduce Julia by translating two Matlab routines. The first example is rather simple and makes Julia shine: it shows that vectorization is not required to make for loops fast. But the second example makes us think a little more about how things are done in Julia.

## 1. First example: make your own matrix-vector product

### Matrix-vector product in Matlab

Computing A*x could not be simpler in Matlab:
```
>>> B = A*x
```
Matlab translates linear algebra operations in suitable calls to BLAS and LAPACK routines. In fact, this is precisely the reason Matlab was created. It is a very convenient wrapper interface for these and other libraries.

But in Matlab you would never write the following:
```
function B = matvec(A, x)
    B = zeros(size(A),1);
    for j = 1:size(A,2)
        for i = 1:size(A,1)
            B(i) = B(i) + A(i,j) * x(j);
        end
    end
end
```
because that would be terribly slow.

### Matrix-vector product in Julia

For linear algebra operations, Julia calls the same BLAS and LAPACK routines as MATLAB. But there is no major performance penalty for writing your own. Let's see how that goes.

Type Shift+Enter to execute each of the code blocks below. This invokes a Just-In-Time (JIT) compiler to compile and execute the code.

In [1]:
function matvec(A, x)
    B = zeros(size(A,1))
    for j = 1:size(A,2)
        for i = 1:size(A,1)
            B[i] = B[i] + A[i,j]*x[j]
        end
    end
    B
end

matvec (generic function with 1 method)

Notice the similarity with Matlab code. One major difference is **indexing with square brackets**, rather than parentheses: `B[i]` instead of `B(i)`. This is for a good reason, by the way. With different notation, the compiler can distinguish between indexing and a function call in all possible cases. Without this distinction at compile-time, you would risk having to do an expensive check at runtime to see what the programmer meant to do. Julia has been designed to help the compiler emit fast code, and this affects syntax decisions too.

Another notable syntax difference: **no semicolons** at the end of each line! (Think about it, what were they for anyway?)

Finally, the Julia routine ends with a line that simply says `B`. Each Julia command has a return value, and the return value of the last line of the function body that is evaluated becomes the return value of the function. So, we are returning `B` as the result of our computation.

Now, how does it compare to `A*x`? First, we verify correctness of our implementation, then we do some timings.

In [15]:
N = 1000
A = rand(N,N)
x = rand(N,1)
B1 = A*x
B2 = matvec(A, x)
maximum(abs.(B1-B2))

6.821210263296962e-13

In [3]:
@time A*x
@time matvec(A,x);

  0.001718 seconds (85 allocations: 14.373 KiB)
  0.001825 seconds (5 allocations: 8.094 KiB)


Not too bad! In my own runs, both timings were very similar. **This means we did not completely kill performance by writing for loops.**

Why would you want to write your own matrix-vector product? Well, you probably don't. Julia is very good at [interfacing](http://julia.readthedocs.org/en/release-0.4/manual/calling-c-and-fortran-code/) to existing software libraries. There is no need to translate everything for the sake of it. The point here is that, at least in principle, you could.

Want to know what the original A\*x command did *exactly*? This is a good way to start your investigation:

In [4]:
@which A*x

*(A::AbstractArray{T,2}, B::AbstractArray{S,2}) where {T, S} in Base.LinAlg at linalg/matmul.jl:145

For now, perhaps you can't make a lot of sense out of the code in `linalg/matmul.jl`. But bear in mind that these are some of the core methods in Julia, and they are written in Julia. The code is there for everyone to see. Pretty soon, there will be no limit to what you can find out about what Julia does with your code. Plus, reading Julia code is a good way to learn.

### A note about timings in Julia

The first time you execute code, the JIT compiler will take some time to compile your source code. This compilation time is included in the timings. That is why in Julia you usually run timings twice, and ignore the first run. Or you perform a warm-up function call: execute the code once before starting the timing. That is what I did above, sneakily: I computed the matrix-vector product once before timing the function, when we were verifying correctness.

Later runs may also give varying results, because Julia performs **garbage collection** in the background, and this is included in the timings also.

A final important remark to make here: commands you type directly at the REPL (Read-Execute-Print-Loop) interface are not optimized. Be sure to place all code inside the body of a function and then time that function.

## 2. Second example: make your own FFT

Our second example is a little more advanced. The syntactical similarities between Julia and Matlab are convenient, but superficial. There is more to it than removing semicolons.

### A simple FFT routine in Matlab

Here is the Matlab way of computing the FFT of a vector:

```
y = fft(x);
```

This is simple and efficient, and it would not be faster in Julia. But why is it fast in Matlab? Like with the matrix-vector product, it is fast because fft is a built-in function. Again, you would never think of writing the following Matlab routine that implements the Cooley-Tukey FFT:

```
function z = my_fft(x)
    N = length(x);
    if N == 1
        z = x;
    else
        z_even = my_fft(x(1:2:end));
        z_odd = my_fft(x(2:2:end));
        z = zeros(size(x));
        for k = 1:N/2
            z(k) = z_even(k) + exp(-2i*pi*(k-1)/N) * z_odd(k);
            z(N/2+k) = z_even(k) - exp(-2i*pi*(k-1)/N) * z_odd(k);
        end
    end
end
```
On my machine, for a vector of length `N=1024`, the routine `my_fft` is slower than `fft` by a factor of a thousand.

There are several reasons why it is slow, some of which are algorithmic and some of which are due to Matlab. First, there is a lot of copying of data going on in the recursive function calls, with statements like `x(1:2:end)`. This seems like an algorithmic problem and I could have coded it differently. But still, the issue is not entirely innocent: do you know of ways in Matlab to modify arguments in-place, or to create views on arrays in order to avoid memory allocation? Are you familiar with Matlab's copy-on-write mechanism?

Second, there is a nasty looking for-loop of length N/2 at the end of the code. Give me a minute, let me vectorize that code because, you know, for loops are slow. Or wait... remind me, why exactly did I think of this for-loop as being *nasty*?

### A simple FFT routine in Julia
Below is a Julia version of my_fft. Again, it looks very similar to the Matlab version.

In [5]:
function my_fft(x)
    N = length(x)
    z = zeros(Complex{Float64}, N)
    if N == 1
        z[1] = x[1]
    else
        z_even = my_fft(x[1:2:end])
        z_odd = my_fft(x[2:2:end])
        Nhalf = N >> 1
        for k = 1:Nhalf
            z[k] = z_even[k] + exp(-2im*pi*(k-1)/N) * z_odd[k]
            z[Nhalf+k] = z_even[k] - exp(-2im*pi*(k-1)/N) * z_odd[k]
        end
    end
    z
end

my_fft (generic function with 1 method)

Let's verify first that our implementation is correct.

In [16]:
a = rand(128)
maximum(abs.(fft(a)-my_fft(a)))

3.4684476073050944e-15

Julia links to the popular FFTW library for these computations. A faster way to use the library is by precomputing a plan and then applying it to a vector:

In [17]:
F = plan_fft(a)
maximum(abs.(F*a - my_fft(a)))

3.4684476073050944e-15

Now, let's have a look at the relative speeds:

In [8]:
a = rand(1024)
@time b1 = fft(a)
F = plan_fft(a)
@time b2 = F*a
@time b3 = my_fft(a);

  0.000116 seconds (67 allocations: 35.875 KiB)
  0.000016 seconds (7 allocations: 32.453 KiB)
  0.000588 seconds (8.19 k allocations: 721.828 KiB)


Our implementation is slower than the external library for sure, but on my machine only by a factor of about 6! It seems that even without specific optimizations, Julia did quite well. Many improvements of our simple implementation are possible to make the performance gap a lot smaller, and in fact a native Julia version of the popular FFTW libary is on the way.

### An incomplete comparison

The Matlab and Julia codes look quite similar, so let's focus on the differences.

First, there are again some minor differences in syntax:
* No semicolons at the end of each line
* Array indexing with square brackets rather than parentheses
* The imaginary unit is `im` instead of `i`: we write `2im` rather than `2i` in the complex exponentials.
* The result of the function is the result of the last line of the function. In this case, we return `z` (or rather, the array that `z` points to), by putting it on an otherwise empty line.

But this time there are two more differences, and they are also more profound. Both are related to a concept unofficially called **type-stability**. This concept is quite pervasive in Julia code. The differences are:
* With the line `z = zeros(Complex{Float64}, N)`, I have allocated memory for the result at the start of the Julia function. It is an array of complex numbers. In Matlab, I wrote `z = zeros(size(x))` only in the `else` branch of the function body. Doing it another way in Julia, I have made sure that the result of my computation is an array of complex numbers, regardless of which branch the code ends up taking and regardless of what happens in the recursive function calls. The type of my output is stable: it is an array of complex numbers. The Julia compiler is smart enough to figure this out, and to use it for subsequent optimizations, even if I didn't mention it explicitly.  
Note that in the Matlab version the type of the output depends on the length of `x`: if `x` is a real vector of length 1, then the line `z=x` causes `z` to be a vector of reals as well. If `x` is a longer vector, the output is complex-valued. **In Julia, best performance is obtained when the type of a return value is uniquely determined by the types of the input arguments.** That is what we call type-stability. Note that the output type may depend only on the types of the input arguments, not on their values. In the case of a standard Julia vector, having an output type depend on the length of a vector violates type-stability, because vectors have variable lengths. Like in C and C++, the length is not part of the type of the vector.
* I have also introduced a new variable with the line `Nhalf = N >> 1`. This is a programmer's trick to quickly divide by 2 using a bitshift. Why did I not write `Nhalf = N/2`? Speed has nothing to do with my decision, it is again due to type-stability. Since I am using Nhalf for indexing later on, Julia requires it to be an integer. But consider the following:

In [9]:
4/2

2.0

In [10]:
typeof(4/2)

Float64

Try the same computation in Matlab and in Python. Unlike those languages, **Julia insists that 4 divided by 2 is 2.0, and not 2**. It is a floating point. If I want the integer 2, I either have to specify it:

In [11]:
Int(4/2)

2

or use the bitshifting trick:

In [12]:
4>>1

2

In [13]:
typeof(4 >> 1)

Int64

Why does 4/2 return a floating point number? This is the reason:

In [14]:
3/2

1.5

Because 3/2 can not be represented by an integer, it has to be a floating point number. In Julia, the division operator `/` is *type-stable*. The types of the numbers I am dividing completely determines the output type, *not their values*. Since Julia can not return an integer when you ask for 3/2, it will not return an integer when you ask for 4/2 either. **The return type of an Int64 divided by an Int64, is a Float64, period. Always.**

There are times when this level of consistency is inconvenient for me, the programmer. But it is of enormous benefit for compiler optimizations. (And it often helps to avoid bugs, too!)

## 3. Is  Julia for you?

Julia code runs faster than Matlab code, that is clear. But I have also made you worry about the types of variables, even if they weren't explicitly mentioned. You usually don't worry about that in Matlab. You usually don't worry about its copy-on-write mechanism either. I also made you question whether 4/2 is an integer. Is it, or is it not? These are some of the idiosyncracies of Julia. Every language has them. Still, in my opinion, there is no denying that **learning Julia is harder than learning Matlab**. That is mainly because it is more powerful. On the other hand, **learning Julia is *orders of magnitude* simpler than learning C++**, even in its modern incarnations.

Julia is not a suitable first language to learn - check out Python. The main motivation for you to try out Julia, coming from a background of Matlab or Python+C, may be its speed. This is a good reason, and the above examples are illustrative. But there are other good reasons that may make you want to stay with Julia. Like with Python and other dynamic languages, Julia code tends to be short and readable. Julia is a very expressive language for mathematics and technical computing, and that makes it often pleasant. Another significant advantage is its open development. If you wonder why something is the way it is, you can often find the answer in an open discussion of the feature in the Github [issues](https://github.com/JuliaLang/julia/issues) or in the [julia-users](https://groups.google.com/forum/#!forum/julia-users) or [julia-dev](https://groups.google.com/forum/#!forum/julia-dev) mailing lists.

In learning Julia, what will be your biggest hurdle? Type-stability may make you worried already, but my bet is on the concept of **multiple dispatch**. We haven't seen it yet in the examples so far, but it is a powerful feature that underlies the efficiency of Julia programs. It leads to a way of programming that is rather different from the typical object-oriented approach you may be more familiar with. Rather, it encourages **generic programming**. This is closer to programming with C++-templates, which perhaps you've seen examples of.

As we mentioned at the top, the goal of these notebooks is to illustrate the point that **the combination of type inference and dynamic multiple dispatch is a great match for technical computing**. This will remain so, regardless of whether the specific language that is Julia will survive in the long term.